In [80]:
import pandas as pd
import geopandas as gpd
# data file from here: https://www.census.gov/geographies/reference-files/time-series/demo/metro-micro/delineation-files.html
df = pd.read_csv('2020_CBSA.csv', dtype={'FIPS County Code': str})



In [81]:
df.columns = df.iloc[1].values#
df.head()#['CSA Title'].value_counts()

,CBSA Code,Metropolitan Division Code,CSA Code,CBSA Title,Metropolitan/Micropolitan Statistical Area,Metropolitan Division Title,CSA Title,County/County Equivalent,State Name,FIPS State Code,FIPS County Code,Central/Outlying County
0,"List 1. CORE BASED STATISTICAL AREAS (CBSAs), ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CBSA Code,Metropolitan Division Code,CSA Code,CBSA Title,Metropolitan/Micropolitan Statistical Area,Metropolitan Division Title,CSA Title,County/County Equivalent,State Name,FIPS State Code,FIPS County Code,Central/Outlying County
2,10100,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Brown County,South Dakota,46,013,Central
3,10100,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Edmunds County,South Dakota,46,045,Outlying
4,10140,NaN,NaN,"Aberdeen, WA",Micropolitan Statistical Area,NaN,NaN,Grays Harbor County,Washington,53,027,Central


In [82]:
df.drop(index=[0,1], inplace=True)

In [83]:
df.loc[df['CSA Code'] == '408'].shape

(31, 12)

In [84]:
nyc = df.loc[df['CSA Code'] == '408']

In [85]:
nyc.head()

,CBSA Code,Metropolitan Division Code,CSA Code,CBSA Title,Metropolitan/Micropolitan Statistical Area,Metropolitan Division Title,CSA Title,County/County Equivalent,State Name,FIPS State Code,FIPS County Code,Central/Outlying County
253,14860,NaN,408,"Bridgeport-Stamford-Norwalk, CT",Metropolitan Statistical Area,NaN,"New York-Newark, NY-NJ-CT-PA",Fairfield County,Connecticut,09,001,Central
552,20700,NaN,408,"East Stroudsburg, PA",Metropolitan Statistical Area,NaN,"New York-Newark, NY-NJ-CT-PA",Monroe County,Pennsylvania,42,089,Central
898,28740,NaN,408,"Kingston, NY",Metropolitan Statistical Area,NaN,"New York-Newark, NY-NJ-CT-PA",Ulster County,New York,36,111,Central
1195,35300,NaN,408,"New Haven-Milford, CT",Metropolitan Statistical Area,NaN,"New York-Newark, NY-NJ-CT-PA",New Haven County,Connecticut,09,009,Central
1208,35620,35004,408,"New York-Newark-Jersey City, NY-NJ-PA",Metropolitan Statistical Area,"Nassau County-Suffolk County, NY","New York-Newark, NY-NJ-CT-PA",Nassau County,New York,36,059,Central


In [86]:
from helper_funcs import *
createDir('NYC_metro_boundaries')

In [87]:
st_tract.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,34,031,256804,34031256804,2568.04,Census Tract 2568.04,G5020,S,29792621,809082,+41.0787934,-074.3779531,"POLYGON ((-74.41591 41.08861, -74.41484 41.089..."
1,34,031,246300,34031246300,2463,Census Tract 2463,G5020,S,10100083,480508,+40.9104210,-074.2642143,"POLYGON ((-74.28932 40.92851, -74.28932 40.928..."
2,34,031,181200,34031181200,1812,Census Tract 1812,G5020,S,401624,0,+40.9242986,-074.1499553,"POLYGON ((-74.15537 40.92394, -74.15535 40.924..."
3,34,031,256803,34031256803,2568.03,Census Tract 2568.03,G5020,S,14231776,740595,+41.1221009,-074.3805752,"POLYGON ((-74.41296 41.11469, -74.41051 41.117..."
4,34,031,246003,34031246003,2460.03,Census Tract 2460.03,G5020,S,6580967,1795588,+40.9770103,-074.2610348,"POLYGON ((-74.28242 40.97003, -74.28241 40.970..."


In [88]:
metro_areas = pd.DataFrame()
for state_cd in nyc['FIPS State Code'].unique():
    st_tract = gpd.read_file(f"https://www2.census.gov/geo/tiger/TIGER2021/TRACT/tl_2021_{state_cd}_tract.zip")
    st_tract.COUNTYFP = st_tract.COUNTYFP.astype(str)
    
    filtered = st_tract.loc[st_tract.COUNTYFP.isin(nyc['FIPS County Code'])]
    
    metro_areas = pd.concat([metro_areas, filtered])
    

In [89]:
state_codes = {
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'PR': '72', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46'
}
codes = dict(zip(state_codes.values(), state_codes.keys()))

In [91]:
metro_areas.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,09,003,514500,09003514500,5145,Census Tract 5145,G5020,S,2673486,48144,+41.7772247,-072.5398572,"POLYGON ((-72.56218 41.78077, -72.56122 41.780..."
1,09,003,514600,09003514600,5146,Census Tract 5146,G5020,S,2144936,0,+41.7692952,-072.5444530,"POLYGON ((-72.55995 41.77014, -72.55994 41.770..."
2,09,003,514700,09003514700,5147,Census Tract 5147,G5020,S,1383464,0,+41.7698536,-072.5227784,"POLYGON ((-72.53319 41.77264, -72.53263 41.772..."
3,09,003,514800,09003514800,5148,Census Tract 5148,G5020,S,1377342,0,+41.7708559,-072.5106023,"POLYGON ((-72.51694 41.77688, -72.51671 41.776..."
4,09,003,514900,09003514900,5149,Census Tract 5149,G5020,S,3546060,22405,+41.7907521,-072.4835364,"POLYGON ((-72.50908 41.79366, -72.50862 41.793..."


In [92]:
metro_areas['state_abbr'] = metro_areas.STATEFP.map(codes)

In [94]:
metro_areas.to_file('nyc_metro_boundaries/nyc_metro_boundaries.shp')

/Users/julialromero/miniconda3/envs/myenv/lib/python3.9/site-packages/geopandas/io/file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [95]:
metro_areas['GEOID'].nunique()

8231